In [1]:
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from os import getenv
from dotenv import load_dotenv

load_dotenv()


True

In [2]:

llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="meta-llama/llama-3.2-3b-instruct:free",
    temperature=0.5,
    max_tokens=100,

)

result = llm.invoke("Hello, how are you today?")
result.content

"I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to help you with any questions or tasks you have. How can I assist you today?"

# String PromptTemplates

In [3]:
from langchain_core.prompts import PromptTemplate
# Create template
pt = PromptTemplate(
    input_variables=["topic"],
    template="Tell me a fact about {topic}."
)

# Use template
formatted_prompt = pt.format(topic="Moon")
print(formatted_prompt)  # Outputs: Tell me a fact about Moon.

Tell me a fact about Moon.


In [4]:
# Define a template for creating prompts that require multiple input variables: 'topic' and 'level'
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"],  # Specify the required input variables
    template="Tell me a fact about {topic} for a student {level} level."  # Template string with placeholders for inputs
)

# Format the prompt by replacing the placeholders in the template with actual values: 'Moon' for topic and '6th Grade' for level
multiple_input_prompt.format(topic='Moon', level='6th Grade')

'Tell me a fact about Moon for a student 6th Grade level.'

In [5]:
result = llm.invoke(multiple_input_prompt.format(topic='Moon', level='6th Grade'))
print(result.content)


Here's a cool fact about the Moon that a 6th-grade student might enjoy:

Did you know that the Moon is actually moving away from the Earth at a rate of about 1.5 inches (3.8 centimeters) every year? This is because the Moon's orbit is slowly increasing in size due to the tidal interactions between the Earth and the Moon. This process is called "tidal acceleration." It's been happening for billions of years, and the Moon is now about 


# ChatPromptTemplates

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

# prompt_template.format(topic="chickens")
p = prompt_template.invoke({"topic": "cats"}).to_messages()
p


[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})]

In [7]:
llm.invoke(p)

AIMessage(content='Why did the cat join a band?\n\nBecause it wanted to be the purr-cussionist.', additional_kwargs={'refusal': ''}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 46, 'total_tokens': 66, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/llama-3.2-3b-instruct', 'system_fingerprint': 'fastcoe', 'finish_reason': 'stop', 'logprobs': None}, id='run-8811a11f-4453-45dd-95c3-ac06bf8b29f1-0', usage_metadata={'input_tokens': 46, 'output_tokens': 20, 'total_tokens': 66, 'input_token_details': {}, 'output_token_details': {}})

# MessagesPlaceholder

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

# Define a chat prompt template with a system message and a placeholder for user messages
prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),  # System message indicating the assistant's role
    MessagesPlaceholder("msgs")  # Placeholder where user messages will be inserted
])

# Invoke the prompt template with a list containing a single human message
p = prompt_template.invoke({"msgs": [HumanMessage(content="hi!")]})  # User sends "hi!" to the assistant

p

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi!', additional_kwargs={}, response_metadata={})])

In [11]:

r = llm.invoke(p)

r.content

'How can I assist you today?'

# Few-Shot PromptTemplates

In [12]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define a list of examples with input and corresponding output
examples = [
    {"input": "2 🦜 2", "output": "4"},
    {"input": "2 🦜 3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("user", "{input}?"),
        ("ai", "{output}.")
    ]
)

In [15]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)
print(few_shot_prompt)

examples=[{'input': '2 🦜 2', 'output': '4'}, {'input': '2 🦜 3', 'output': '5'}] input_variables=[] input_types={} partial_variables={} example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['output'], input_types={}, partial_variables={}, template='{output}.'), additional_kwargs={})])


In [16]:
p = few_shot_prompt.invoke({}).to_messages()

p

[HumanMessage(content='2 🦜 2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='2 🦜 3?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5.', additional_kwargs={}, response_metadata={})]

In [17]:

r = llm.invoke(p)
r.content


"You're using emojis to represent numbers. I'll follow along.\n\n2 birds (🦜) 2 = 4\n2 birds (🦜) 3 = 5"